<a href="https://colab.research.google.com/github/eemayes2/IE517_F21_HW6/blob/main/IE517_HWK6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
#Import libraries needed
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#Check if any null values we need to change
def num_missing(x):
    return sum(x.isnull())

In [49]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [50]:
#Read in Data
df = pd.read_csv('gdrive/MyDrive/Colab Notebooks/ccdefault.csv', header=0)
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT
0,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [51]:
df.describe()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000
mean,15000.500000,167484.322667,1.603733,1.853133,1.551867,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,-0.266200,-0.291100,51223.330900,49179.075167,4.701315e+04,43262.948967,40311.400967,38871.760400,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.221200
std,8660.398374,129747.661567,0.489129,0.790349,0.521970,9.217904,1.123802,1.197186,1.196868,1.169139,1.133187,1.149988,73635.860576,71173.768783,6.934939e+04,64332.856134,60797.155770,59554.107537,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.415062
min,1.000000,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-165580.000000,-69777.000000,-1.572640e+05,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,7500.750000,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3558.750000,2984.750000,2.666250e+03,2326.750000,1763.000000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,15000.500000,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22381.500000,21200.000000,2.008850e+04,19052.000000,18104.500000,17071.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,22500.250000,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67091.000000,64006.250000,6.016475e+04,54506.000000,50190.500000,49198.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,0.000000
max,30000.000000,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,964511.000000,983931.000000,1.664089e+06,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


### Try different random states in test-train-split

In [52]:
#Split into training-test sets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
x = df.drop(columns = ['DEFAULT'])
y = df['DEFAULT']
out_of = []
in_of = []
for i in range(1,11):
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=i, stratify = y)
  tree = DecisionTreeClassifier(criterion = 'gini', max_depth = 4, random_state = 1)
  tree.fit(X_train, y_train)
  y_pred = tree.predict(X_test)
  acc_out = accuracy_score(y_test, y_pred)
  out_of.append(acc_out)
  y_pred_train = tree.predict(X_train)
  acc_in = accuracy_score(y_train, y_pred_train)
  in_of.append(acc_in)
  print("Seed Number:" + str(i) + "\n\tOut of Sample Accuracy: " + str(acc_out))
  print("\tIn Sample Accuracy: " + str(acc_in))

Seed Number:1
	Out of Sample Accuracy: 0.8283333333333334
	In Sample Accuracy: 0.8225555555555556
Seed Number:2
	Out of Sample Accuracy: 0.824
	In Sample Accuracy: 0.8225925925925925
Seed Number:3
	Out of Sample Accuracy: 0.8173333333333334
	In Sample Accuracy: 0.8241111111111111
Seed Number:4
	Out of Sample Accuracy: 0.8196666666666667
	In Sample Accuracy: 0.8237037037037037
Seed Number:5
	Out of Sample Accuracy: 0.818
	In Sample Accuracy: 0.8231481481481482
Seed Number:6
	Out of Sample Accuracy: 0.819
	In Sample Accuracy: 0.823925925925926
Seed Number:7
	Out of Sample Accuracy: 0.8253333333333334
	In Sample Accuracy: 0.8232592592592592
Seed Number:8
	Out of Sample Accuracy: 0.8166666666666667
	In Sample Accuracy: 0.8238148148148148
Seed Number:9
	Out of Sample Accuracy: 0.8173333333333334
	In Sample Accuracy: 0.823925925925926
Seed Number:10
	Out of Sample Accuracy: 0.82
	In Sample Accuracy: 0.8236296296296296


In [53]:
print("Out of Sample: \n\tMean: " + str(np.mean(out_of)) + "\n\tStand. Dev: " + str(np.std(out_of)))
print("In Sample: \n\tMean: " + str(np.mean(in_of)) + "\n\tStand. Dev: " + str(np.std(in_of)))


Out of Sample: 
	Mean: 0.8205666666666666
	Stand. Dev: 0.0037566237797019685
In Sample: 
	Mean: 0.8234666666666666
	Stand. Dev: 0.0005272284340525735


### Cross Validation with K-fold CV

In [54]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
x = df.drop(columns = ['DEFAULT'])
y = df['DEFAULT']

cv = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
scores = cross_val_score(tree, x, y, scoring='accuracy', cv=cv)

scores

array([0.819     , 0.827     , 0.82366667, 0.82466667, 0.82033333,
       0.817     , 0.81233333, 0.81833333, 0.81366667, 0.82566667])

In [55]:
print("Cross Val Scores: \n\tMean: " + str(np.mean(scores)) + "\n\tStand. Dev: " + str(np.std(scores)))


Cross Val Scores: 
	Mean: 0.8201666666666666
	Stand. Dev: 0.00477318901085357


In [56]:
#Out of sample accuracy
kf = StratifiedKFold(n_splits=10, random_state = 1, shuffle = True)
kf.get_n_splits(x,y)
i = 1 
out_of = []

for train_index, test_index in kf.split(x,y):
  X_train, X_test = x.iloc[list(train_index)], x.iloc[list(test_index)]
  y_train, y_test = y.iloc[list(train_index)], y.iloc[list(test_index)]
  tree.fit(X_train, y_train)
  y_pred = tree.predict(X_test)
  acc_out = accuracy_score(y_test, y_pred)
  out_of.append(acc_out)
  y_pred_train = tree.predict(X_train)
  print("Fold Index:" + str(i) + "\n\tOut of Sample Accuracy: " + str(acc_out))
  print("\tIn Sample Accuracy: " + str(acc_in))
  i += 1


Fold Index:1
	Out of Sample Accuracy: 0.819
	In Sample Accuracy: 0.8236296296296296
Fold Index:2
	Out of Sample Accuracy: 0.827
	In Sample Accuracy: 0.8236296296296296
Fold Index:3
	Out of Sample Accuracy: 0.8236666666666667
	In Sample Accuracy: 0.8236296296296296
Fold Index:4
	Out of Sample Accuracy: 0.8246666666666667
	In Sample Accuracy: 0.8236296296296296
Fold Index:5
	Out of Sample Accuracy: 0.8203333333333334
	In Sample Accuracy: 0.8236296296296296
Fold Index:6
	Out of Sample Accuracy: 0.817
	In Sample Accuracy: 0.8236296296296296
Fold Index:7
	Out of Sample Accuracy: 0.8123333333333334
	In Sample Accuracy: 0.8236296296296296
Fold Index:8
	Out of Sample Accuracy: 0.8183333333333334
	In Sample Accuracy: 0.8236296296296296
Fold Index:9
	Out of Sample Accuracy: 0.8136666666666666
	In Sample Accuracy: 0.8236296296296296
Fold Index:10
	Out of Sample Accuracy: 0.8256666666666667
	In Sample Accuracy: 0.8236296296296296


In [57]:
print("Out of Sample: \n\tMean: " + str(np.mean(out_of)) + "\n\tStand. Dev: " + str(np.std(out_of)))


Out of Sample: 
	Mean: 0.8201666666666666
	Stand. Dev: 0.00477318901085357


In [58]:
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('IE517_HWK6.ipynb')

File ‘colab_pdf.py’ already there; not retrieving.

Mounted at /content/drive/




[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/IE517_HWK6.ipynb to pdf
[NbConvertApp] Writing 38733 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: [u'xelatex', u'./notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: [u'bibtex', u'./notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 35670 bytes to /content/drive/My Drive/IE517_HWK6.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'File ready to be Downloaded and Saved to Drive'